<a href="https://colab.research.google.com/github/yiw008/nondet-project/blob/main/Go_Through_IFixFlakies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import requests
import csv

In [ ]:
# URL of the CSV file: iPFlakies
url = "https://zenodo.org/records/6176417/files/Test_Status.csv"

In [ ]:
# Read the CSV file from the URL
data = pd.read_csv(url)

In [ ]:
# Display the first few rows of the data
print(data.head())

  Project_Name                               Project_URL  \
0   BT-Tracker  https://github.com/nordwind80/BT-Tracker   
1      Breathe         https://github.com/mrob95/Breathe   
2      Breathe         https://github.com/mrob95/Breathe   
3      Breathe         https://github.com/mrob95/Breathe   
4      Breathe         https://github.com/mrob95/Breathe   

                               Project_Hash  \
0  558c15b399871c1ca11d0c4ae1eb598e3060931e   
1  4600818e24f4156cd7bb8cc0f43886b27323968e   
2  4600818e24f4156cd7bb8cc0f43886b27323968e   
3  4600818e24f4156cd7bb8cc0f43886b27323968e   
4  4600818e24f4156cd7bb8cc0f43886b27323968e   

                                             Test_id  Detected  Have_Patch  \
0  Tracker/tests/test_event.py::TestEvent::test_o...      True        True   
1  tests/test_command_context.py::test_manual_con...      True       False   
2                tests/test_loading.py::test_loading      True        True   
3        tests/test_loading.py::test_loading

In [ ]:
def match_class(content, class_name):
  start_index = content.find(f"class {class_name}")
  if start_index == -1:
    return None
  return content[start_index:]

In [ ]:
def match_method(content, test_name):
  start_index = content.find(f"def {test_name}")
  if start_index == -1:
    return None

  end_index = -1
  next_def_index = content.find("def ", start_index + len(test_name))
  main_block_index = content.find("if __name__ == '__main__':", start_index)

  if next_def_index != -1:
    end_index = next_def_index
  elif main_block_index != -1:
    end_index = main_block_index
  else:
    end_index = len(content)

  return content[start_index:end_index]

In [ ]:
def remove_square_brackets(test_name):
  start_index = test_name.find('[')
  end_index = test_name.find(']')

  if start_index != -1 and end_index != -1:
    new_test_name = test_name[:start_index]
    return new_test_name
  else:
    return test_name

In [ ]:
def read_python_file(url):
  try:
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP errors
    # The content of the file is in response.text
    python_code = response.text
    return python_code
  except requests.exceptions.RequestException as e:
    print(f"Error fetching the file: {e}")
    return None

In [ ]:
def special_cases(project_name, repo_url, commit_hash, test_file_path, class_name, test_name):
  if project_name == 'Butter.MAS.PythonAPI' and commit_hash == 'f86ebe75df3826f62a268645cdbe4400b43fab07' and test_file_path.startswith('butter/mas/tests/clients/'):
    test_file_path = 'butter/mas/tests/clients/client_test.py'
    class_name = ''

  if project_name == 'SNData' and commit_hash == 'e4854f0dc357484b437b15f9dac15f7c589eff58' and (class_name == 'Sako18Parsing' or class_name == 'DR1Parsing'):
    test_file_path = 'tests/data_parsing_template_tests.py'
    class_name = ''
    if test_name == 'test_standard_column_names':
      class_name = 'PhotometricDataParsing'

  if project_name == 'bootstrap_env' and commit_hash == 'ab68025d8f6b9a17d8feeed83e8aae26e3f28769' and test_file_path.startswith('bootstrap_env/tests/'):
    test_file_path = 'bootstrap_env/tests/base.py'

  if repo_url.endswith('.git'):
    repo_url = repo_url[:-4]

  if project_name == 'data-pypes' and test_name == 'pypes.logsetup.get_logconfig':
    test_name = 'get_logconfig'

  if project_name == 'elife-tools':
    end_index = test_name.find('_1_elife_02833_v2_xml')
    test_name = test_name[:end_index]

  if project_name == 'noipy':
    repo_url = 'https://github.com/pv8/noipy'

  if project_name == 'pymq' and commit_hash == '101857bca2b705c328d3bda3b26797b51e8ffb70':
    if class_name == 'SimplePubSubTest':
      test_file_path = 'tests/base/pubsub.py'
      class_name = 'AbstractPubSubTest'
    if class_name == 'IpcQueueTest':
      test_file_path = 'tests/base/queue.py'
      class_name = 'AbstractQueueTest'
    if class_name == 'IpcRpcTest' or class_name == 'SimpleRpcTest':
      test_file_path = 'tests/base/rpc.py'
      class_name = 'AbstractRpcTest'

  if project_name == 'pyswarms' and commit_hash == '08756526f39699eef28e515cac2ead17cef55710' and class_name == 'TestLocalBestOptimizer' and test_name == 'test_obj_with_kwargs':
    test_file_path = 'tests/optimizers/abc_test_optimizer.py'
    class_name = 'ABCTestOptimizer'

  if project_name == 'python-openflow' and commit_hash == 'a3387a7b28d529a3605aa1506a028e03394e4526' and class_name == 'TestFlowMod' and test_name == 'test_minimum_size':
    test_file_path = 'tests/unit/test_struct.py'
    class_name = 'TestStruct'

  return project_name, repo_url, commit_hash, test_file_path, class_name, test_name

In [ ]:
all_test_methods = []
print('The indices below corresponds to the row number in the csv file shown on https://sites.google.com/view/ipflakies.')

for index, row in data.iterrows():
  row_in_csv = index + 2
  project_name = row['Project_Name']
  repo_url = row['Project_URL']
  commit_hash = row['Project_Hash']
  if len(row['Test_id'].split('::')) > 2:
    test_file_path = row['Test_id'].split('::')[0]
    class_name = row['Test_id'].split('::')[1]
    test_name = row['Test_id'].split('::')[2]
  else:
    test_file_path = row['Test_id'].split('::')[0]
    class_name = ''
    test_name = row['Test_id'].split('::')[1]
  test_name = remove_square_brackets(test_name)

  test_file_path_rec = test_file_path
  class_name_rec = class_name
  test_name_rec = test_name

  project_name, repo_url, commit_hash, test_file_path, class_name, test_name = special_cases(project_name, repo_url, commit_hash, test_file_path, class_name, test_name)

  new_url = repo_url.replace('github.com', 'raw.githubusercontent.com')
  new_url += '/' + commit_hash + '/' + test_file_path

  content = read_python_file(new_url)
  if content is None:
    print(f"Row {row_in_csv}: {repo_url}/blob/{commit_hash}/{test_file_path}, Error fetching the file\n")
    continue

  # Class content
  if class_name != '':
    content = match_class(content, class_name)
    if content is None:
      print(f"Row {row_in_csv}: In {repo_url}/blob/{commit_hash}/{test_file_path}, Class '{class_name}' not found.")
      continue

  # Test method content
  content = match_method(content, test_name)
  if content is None:
    print(f"Row {row_in_csv}: In {repo_url}/blob/{commit_hash}/{test_file_path}, Class '{class_name}' , Test function '{test_name}' not found.")
    continue

  test_method = {}
  test_method['Row'] = row_in_csv
  test_method['URL'] = repo_url + '/blob/' + commit_hash + '/' + test_file_path_rec
  test_method['Class'] = class_name_rec
  test_method['Test'] = test_name_rec
  test_method['Content'] = content
  all_test_methods.append(test_method)

The indices below corresponds to the row number in the csv file shown on https://sites.google.com/view/ipflakies.
Error fetching the file: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/AshtonUPS/Py-MI-PS/2d22327c75bac1b58a4804a61e7a703ecc5ba978/src/PyMIPS/tests/register_test.py
Row 135: https://github.com/AshtonUPS/Py-MI-PS/blob/2d22327c75bac1b58a4804a61e7a703ecc5ba978/src/PyMIPS/tests/register_test.py, Error fetching the file

Error fetching the file: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/cryptowatch/cw-sdk-python/92bd90db16dfc116c0708d19d27208d9bfc990c1/tests/test_api.py
Row 516: https://github.com/cryptowatch/cw-sdk-python/blob/92bd90db16dfc116c0708d19d27208d9bfc990c1/tests/test_api.py, Error fetching the file

Error fetching the file: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/cryptowatch/cw-sdk-python/92bd90db16dfc116c0708d19d27208d9bfc990c1/tests/test_api.py
Row 517: https://github.com/cryptowat

In [ ]:
all_test_methods[0]

{'Row': 2,
 'URL': 'https://github.com/nordwind80/BT-Tracker/blob/558c15b399871c1ca11d0c4ae1eb598e3060931e/Tracker/tests/test_event.py',
 'Class': 'TestEvent',
 'Test': 'test_object',
 'Content': 'def test_object(self):\n        assert id(status.state) == id(status.state)\n\n    '}

In [ ]:
len(all_test_methods)

1932

In [ ]:
filename = "all_test_methods.csv"
with open(filename, mode="w", newline="") as file:
  writer = csv.DictWriter(file, fieldnames=all_test_methods[0].keys())
  writer.writeheader()
  writer.writerows(all_test_methods)